In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch
import pandas as pd
from datasets import Dataset
import numpy as np
from transformers import pipeline
from transformers_interpret import MultiLabelClassificationExplainer

# Interpreting the model trained on fake data and tested on real data

In [2]:
dataset_filename = "../preprocessing/data/processed_merge.csv"
# dataset_filename = "../linguistic_model/data/synthetic_clan_train_new.csv"
data = pd.read_csv(dataset_filename, encoding='utf8', index_col=False)

In [3]:
model_name = "models/bert_finetuned_reverse"
tokenizer_name = "models/bert_finetuned_reverse"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, return_tensor="pt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("text-classification", model=model_name, tokenizer=tokenizer_name, device=device)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

In [4]:
model.config.id2label

{0: 'CONTROL', 1: 'BROCA'}

In [5]:
dataset = Dataset.from_pandas(data)
def preprocess_function(examples):
    return tokenizer(examples["preprocessed_text"], padding=True, truncation=True, return_tensors="pt")
tokenized_data = dataset.map(preprocess_function, batched=True).with_format("torch")
tokenized_data_split = tokenized_data.train_test_split(test_size=0.2, seed=42)

Map:   0%|          | 0/57051 [00:00<?, ? examples/s]

In [6]:
num = 0
n = 10
for text, label in zip(list(tokenized_data_split["test"]["preprocessed_text"]), np.array(tokenized_data_split["test"]["label"])): 
    if num != n:
        print("True: ", model.config.id2label[label], "Pred: ", pipe(text)[0]["label"])
        print("Text: ", text)
        cls_explainer(text)
        cls_explainer.visualize()
        print("----------------------------------------------------------------------")
        num += 1

True:  CONTROL Pred:  CONTROL
Text:  i usually use a spoon for my jelly.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.98),CONTROL,1.74,[CLS] i usually use a spoon for my jelly . [SEP]
n/a,(0.03),BROCA,-1.97,[CLS] i usually use a spoon for my jelly . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  the one didn't take the umbrella.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.67),CONTROL,1.11,[CLS] the one didn ' t take the umbrella . [SEP]
n/a,(0.40),BROCA,-0.98,[CLS] the one didn ' t take the umbrella . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  get a scoop with that.


...

----------------------------------------------------------------------
True:  BROCA Pred:  BROCA
Text:  and they one one to.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.03),CONTROL,-1.91,[CLS] and they one one to . [SEP]
n/a,(0.95),BROCA,1.94,[CLS] and they one one to . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  you spread that on the other piece of bread.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.98),CONTROL,1.32,[CLS] you spread that on the other piece of bread . [SEP]
n/a,(0.04),BROCA,-1.51,[CLS] you spread that on the other piece of bread . [SEP]


----------------------------------------------------------------------


# Interpreting the model trained on real data and tested on fake data

In [7]:
# dataset_filename = "../preprocessing/data/processed_merge.csv"
dataset_filename = "../linguistic_model/data/synthetic_clan_train_new.csv"
data = pd.read_csv(dataset_filename, encoding='utf8', index_col=False)

In [8]:
model_name = "models/bert_finetuned"
tokenizer_name = "models/bert_finetuned"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, return_tensor="pt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("text-classification", model=model_name, tokenizer=tokenizer_name, device=device)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

In [9]:
model.config.id2label

{0: 'CONTROL', 1: 'BROCA'}

In [10]:
dataset = Dataset.from_pandas(data)
def preprocess_function(examples):
    return tokenizer(examples["modified"], padding=True, truncation=True, return_tensors="pt")
tokenized_data = dataset.map(preprocess_function, batched=True).with_format("torch")
tokenized_data_split = tokenized_data.train_test_split(test_size=0.2, seed=42)

Map:   0%|          | 0/49850 [00:00<?, ? examples/s]

In [11]:
num = 0
n = 10
for text, label in zip(list(tokenized_data_split["test"]["modified"]), np.array(tokenized_data_split["test"]["label"])): 
    if num != n:
        print("True: ", model.config.id2label[label], "Pred: ", pipe(text)[0]["label"])
        print("Text: ", text)
        cls_explainer(text)
        cls_explainer.visualize()
        print("----------------------------------------------------------------------")
        num += 1

True:  CONTROL Pred:  CONTROL
Text:  i love documentaries.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.65),BROCA,1.26,[CLS] i love documentaries . [SEP]
n/a,(0.37),CONTROL,-1.29,[CLS] i love documentaries . [SEP]


----------------------------------------------------------------------
True:  BROCA Pred:  CONTROL
Text:  plods on managing to get my in harts.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.77),BROCA,1.38,[CLS] pl ##ods on managing to get my in hart ##s . [SEP]
n/a,(0.25),CONTROL,-1.35,[CLS] pl ##ods on managing to get my in hart ##s . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  murder by numbers does not take place in a single room,like the rope,mind you.and,what a supreme originality,it pits two cops against the evil youngstersand,you would never guess it,these two cops are very differentactually,bullock plays the part of woman living like a man,and her partner chaplin is as shy as a clueless girlie.the two boys' performances are not really mind boggling,not as good,as,say,that of edward norton in primal fear.


...

----------------------------------------------------------------------
True:  BROCA Pred:  CONTROL
Text:  ten reviews, handfuls are.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.79),BROCA,1.25,"[CLS] ten reviews , handful ##s are . [SEP]"
n/a,(0.24),CONTROL,-1.23,"[CLS] ten reviews , handful ##s are . [SEP]"


----------------------------------------------------------------------
True:  CONTROL Pred:  BROCA
Text:  coach preachy or straight sappy.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.51),BROCA,0.55,[CLS] coach preach ##y or straight sap ##py . [SEP]
n/a,(0.51),CONTROL,-0.40,[CLS] coach preach ##y or straight sap ##py . [SEP]


----------------------------------------------------------------------
